# Efficient AD Model For Anomaly Detection

## Introduction

## Before running the notebook

Let's prepare the environement before running the notebook.  
Run the following command in the terminal from the **`final`** folder.

```sh
conda create -p envs/2_efficientad python=3.9.7 -y
conda activate envs/2_efficientad

pip install -r requirements/2_efficientad.txt
```

## Import

In [1]:
# # Saving channel_mean and channel_std as a .pth file  
# torch.save({'mean': channel_mean, 'std': channel_std}, 'normalization_params.pth')
# # Loading the parameters back  
# params = torch.load('normalization_params.pth')  
# channel_mean = params['mean']  
# channel_std = params['std']  

In [2]:
import sys
sys.path.append('../code/')

from efficientad import *

In [3]:
config = {
    "seed": 42,
    "on_gpu": torch.cuda.is_available(),
    "out_channels": 384,
    "image_size": 256,
    "dataset_path": "./my_dataset",
    "subdataset": "cookies",
    "output_dir": "../output/1",
    "train_steps": 20,
}

## EfficientAD

In [4]:
set_seed(config["seed"])

Seed set (42)


In [5]:
train_output_dir = os.path.join(config["output_dir"], 'trainings', config["subdataset"])
test_output_dir = os.path.join(config["output_dir"], 'anomaly_maps', config["subdataset"], 'test')

In [6]:
transforms_class = ImageTransforms(config["image_size"])  

full_train_set = ImageFolderWithoutTarget(
    os.path.join(config["dataset_path"], config["subdataset"], 'train'),
    transform=transforms.Lambda(transforms_class.train_transform))

test_set = ImageFolderWithPath(
    os.path.join(config["dataset_path"], config["subdataset"], 'test'))

In [7]:
# mvtec dataset paper recommend 10% validation set
train_size = int(0.9 * len(full_train_set))
validation_size = len(full_train_set) - train_size
rng = torch.Generator().manual_seed(config["seed"])
train_set, validation_set = torch.utils.data.random_split(full_train_set,
                                                    [train_size,
                                                    validation_size],
                                                    rng)

In [8]:
train_loader = DataLoader(train_set, batch_size=1, shuffle=True,
                            num_workers=4, pin_memory=True)
train_loader_infinite = InfiniteDataloader(train_loader)
validation_loader = DataLoader(validation_set, batch_size=1)

In [9]:
penalty_loader_infinite = itertools.repeat(None)
teacher = get_pdn_small(config["out_channels"])
student = get_pdn_small(2 * config["out_channels"])
autoencoder = get_autoencoder(config["out_channels"])

In [10]:
 # teacher frozen
teacher.eval()
student.train()
autoencoder.train()
print("Training")

if config["on_gpu"]:
    teacher.cuda()
    student.cuda()
    autoencoder.cuda()

Training


In [11]:
params = torch.load('../output/normalization_params.pth')  
teacher_mean = params['mean']  
teacher_std = params['std']  

# teacher_mean, teacher_std = teacher_normalization(teacher, train_loader, config)
# torch.save({'mean': teacher_mean, 'std': teacher_std}, f'../output/normalization_params.pth')  

In [12]:
optimizer = torch.optim.Adam(
    itertools.chain(student.parameters(),
    autoencoder.parameters()),
    lr=1e-4, weight_decay=1e-5
)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(0.95 * config["train_steps"]), gamma=0.1)
tqdm_obj = tqdm(range(config["train_steps"]))

  0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
for iteration, (image_st, image_ae), image_penalty in zip(
        tqdm_obj, train_loader_infinite, penalty_loader_infinite):
    if config["on_gpu"]:
        image_st = image_st.cuda()
        image_ae = image_ae.cuda()
    with torch.no_grad():
        teacher_output_st = teacher(image_st)
        teacher_output_st = (teacher_output_st - teacher_mean) / teacher_std
    student_output_st = student(image_st)[:, :config["out_channels"]]
    distance_st = (teacher_output_st - student_output_st) ** 2
    d_hard = torch.quantile(distance_st, q=0.999)
    loss_hard = torch.mean(distance_st[distance_st >= d_hard])
    loss_st = loss_hard
    
    ae_output = autoencoder(image_ae)
    
    with torch.no_grad():
        teacher_output_ae = teacher(image_ae)
        teacher_output_ae = (teacher_output_ae - teacher_mean) / teacher_std
    student_output_ae = student(image_ae)[:, config["out_channels"]:]
    distance_ae = (teacher_output_ae - ae_output)**2
    distance_stae = (ae_output - student_output_ae)**2
    loss_ae = torch.mean(distance_ae)
    loss_stae = torch.mean(distance_stae)
    loss_total = loss_st + loss_ae + loss_stae
    optimizer.zero_grad()
    loss_total.backward()
    optimizer.step()
    scheduler.step()
    
    if iteration % 10 == 0:
        tqdm_obj.set_description(
            "Current loss: {:.4f}  ".format(loss_total.item()))

    if iteration % 1000 == 0:
        torch.save(teacher, os.path.join(train_output_dir,
                                            'teacher_tmp.pth'))
        torch.save(student, os.path.join(train_output_dir,
                                            'student_tmp.pth'))
        torch.save(autoencoder, os.path.join(train_output_dir,
                                                'autoencoder_tmp.pth'))

    if iteration % 10000 == 0 and iteration > 0:
        # run intermediate evaluation
        teacher.eval()
        student.eval()
        autoencoder.eval()

        q_st_start, q_st_end, q_ae_start, q_ae_end = map_normalization(
            validation_loader=validation_loader, teacher=teacher,
            student=student, autoencoder=autoencoder,
            teacher_mean=teacher_mean, teacher_std=teacher_std,
            desc='Intermediate map normalization')
        auc = test(
            test_set=test_set, teacher=teacher, student=student,
            autoencoder=autoencoder, teacher_mean=teacher_mean,
            teacher_std=teacher_std, q_st_start=q_st_start,
            q_st_end=q_st_end, q_ae_start=q_ae_start, q_ae_end=q_ae_end,
            test_output_dir=None, desc='Intermediate inference')
        print('Intermediate image auc: {:.4f}'.format(auc))

        # teacher frozen
        teacher.eval()
        student.train()
        autoencoder.train()

Current loss: 29.3504  : 100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


In [14]:
teacher.eval()
student.eval()
autoencoder.eval()
print("Eval done")

Eval done


In [15]:
torch.save(teacher_mean, os.path.join(train_output_dir, 'teacher_mean.pth'))
torch.save(teacher_std, os.path.join(train_output_dir, 'teacher_std.pth'))
torch.save(teacher, os.path.join(train_output_dir, 'teacher_final.pth'))
torch.save(student, os.path.join(train_output_dir, 'student_final.pth'))
torch.save(autoencoder, os.path.join(train_output_dir,'autoencoder_final.pth'))

In [17]:
 q_st_start, q_st_end, q_ae_start, q_ae_end = map_normalization(config=config,
    validation_loader=validation_loader, teacher=teacher, student=student,
    autoencoder=autoencoder, teacher_mean=teacher_mean,
    teacher_std=teacher_std, desc='Final map normalization')
auc = test(config=config,default_transform=transforms_class.default_transform,
    test_set=test_set, teacher=teacher, student=student,
    autoencoder=autoencoder, teacher_mean=teacher_mean,
    teacher_std=teacher_std, q_st_start=q_st_start, q_st_end=q_st_end,
    q_ae_start=q_ae_start, q_ae_end=q_ae_end,
    test_output_dir=test_output_dir, desc='Final inference')
print('Final image auc: {:.4f}'.format(auc))

Final inference: 100%|██████████| 120/120 [00:20<00:00,  5.86it/s]

Final image auc: 73.9000
